In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
class GoogleCrawlerSelenium():
    def __init__(self,limit):
        self.links_lista = []
        self.texto_lista = []
        self.limit = limit
        self.df = None

    # Busca pelo site google
    def searchInDrive(self,query):
        driver = webdriver.Firefox(executable_path="./geckodriver")
        driver.get("https://google.com.br/")
        
        elem = driver.find_element_by_name("q") # Busca pelo botão de pesquisa
        elem.clear()
        elem.send_keys(query) # Busca a ser realizada no site
        time.sleep(5)
        driver.find_element_by_class_name("gNO89b").click() # Acionar o bota da busca

        
        for i in range(self.limit):
            time.sleep(5)
            links = driver.find_elements_by_xpath("//div[contains(@class,'yuRUbf')]")
            for link in links:
                self.links_lista.append(link.find_element_by_xpath('.//a').get_attribute('href'))
                self.texto_lista.append(link.find_element_by_xpath('.//a/h3/span').text)
            try:
                driver.find_element_by_id("pnnext").click()
            except:
                self.df = pd.DataFrame({"links" : self.links_lista, "titulo" : self.texto_lista})
                print("Numero excedido e dataframe criado")
                break
        
        driver.close() # Fecha o drive após finalizar tudo


In [3]:
crawler = GoogleCrawlerSelenium(30)
crawler.searchInDrive("coronavirus perguntas e respostas")

Numero excedido e dataframe criado


In [4]:
df = crawler.df
df.to_csv('./scraped_links_senelium.csv',index = False)

In [5]:
df

,links,titulo
0,https://coronavirus.saude.gov.br/index.php/per...,Perguntas e respostas - Coronavírus - Ministér...
1,https://portal.fiocruz.br/coronavirus/pergunta...,Coronavírus: Perguntas e respostas - Fundação ...
2,https://www.unimed.coop.br/viver-bem/saude-em-...,Coronavírus e COVID-19: perguntas e respostas ...
3,https://www.hospitalinfantilsabara.org.br/perg...,Perguntas e respostas sobre o novo Coronavírus...
4,https://g1.globo.com/bemestar/coronavirus/noti...,Coronavírus: veja perguntas e respostas | Coro...
...,...,...
137,https://agenciabrasil.ebc.com.br/saude/noticia...,Agência Brasil esclarece dúvidas sobre o coron...
138,https://openwho.org/courses/introducao-ao-ncov,"Vírus respiratórios emergentes, incluindo COVI..."
139,https://books.google.com.br/books?id=03jeDwAAQ...,Coronavírus e os Impactos Trabalhistas: Direit...
140,https://books.google.com.br/books?id=SOQJEAAAQ...,Escola amazônica de liderança: Uma história nã...


# Data análise

In [21]:
df_crawler = pd.read_csv('./scraped_links_senelium.csv')
df_reposi = pd.read_csv('./links_repositorio.csv')

In [23]:
df_reposi.drop(['posicao'],axis=1,inplace=True)
df_reposi

,link
0,https://portal.fiocruz.br/coronavirus/pergunta...
1,https://covid19.min-saude.pt/perguntas-frequen...
2,https://coronavirus.saude.gov.br/index.php/per...
3,http://postalsaude.com.br/beneficiario/noticia...
4,https://www.saude.ma.gov.br/perguntas-e-respos...
5,http://www.saude.ba.gov.br/temasdesaude/corona...
6,http://www.saude.sp.gov.br/ses/perfil/cidadao/...
7,http://portal.anvisa.gov.br/web/guest/noticias...
8,http://www.coronavirus.pr.gov.br/Campanha/FAQ/...
9,https://prefeitura.rio/saude/novo-coronavirus-...


In [24]:
df_crawler.duplicated().sum()

0

In [28]:
aux = []
for link_repo in df_reposi['link']:
    if(link_repo in df_crawler['links'].values):
        aux.append(link_repo)

removed_links = []    
for i in range(len(df_crawler)):
    if(df_crawler.links.values[i] in aux):
        removed_links.append(i)
df_crawler.drop(df_crawler.index[[removed_links]],inplace=True)       

/home/p1ntsu/anaconda3/envs/crawler/lib/python3.8/site-packages/pandas/core/indexes/base.py:4111: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [29]:
df_crawler.reset_index(inplace=True,drop=True)
df_crawler

,links,titulo
0,https://www.unimed.coop.br/viver-bem/saude-em-...,Coronavírus e COVID-19: perguntas e respostas ...
1,https://www.hospitalinfantilsabara.org.br/perg...,Perguntas e respostas sobre o novo Coronavírus...
2,https://g1.globo.com/bemestar/coronavirus/noti...,Coronavírus: veja perguntas e respostas | Coro...
3,https://coronavirus.saude.gov.br/linha-do-tempo/,NaN
4,https://aps.saude.gov.br/ape/corona/telesus,NaN
...,...,...
127,https://agenciabrasil.ebc.com.br/saude/noticia...,Agência Brasil esclarece dúvidas sobre o coron...
128,https://openwho.org/courses/introducao-ao-ncov,"Vírus respiratórios emergentes, incluindo COVI..."
129,https://books.google.com.br/books?id=03jeDwAAQ...,Coronavírus e os Impactos Trabalhistas: Direit...
130,https://books.google.com.br/books?id=SOQJEAAAQ...,Escola amazônica de liderança: Uma história nã...


In [30]:
for i in df_crawler['links']:
    print(i)

https://www.unimed.coop.br/viver-bem/saude-em-pauta/coronavirus-e-covid-19-perguntas-e-respostas
https://www.hospitalinfantilsabara.org.br/perguntas-e-respostas-sobre-o-novo-coronavirus/
https://g1.globo.com/bemestar/coronavirus/noticia/2020/02/27/coronavirus-veja-perguntas-e-respostas.ghtml
https://coronavirus.saude.gov.br/linha-do-tempo/
https://aps.saude.gov.br/ape/corona/telesus
https://bvsms.saude.gov.br/ultimas-noticias/3135-novo-coronavirus-covid-19-informacoes-basicas
https://coronavirus.saude.gov.br/sobre-a-doenca
https://medprev.online/blog/doencas/perguntas-e-respostas-sobre-coronavrus-tire-todas-as-dvidas-sobre-pandemia/
https://www.ladoaladopelavida.org.br/detalhe-noticia-ser-informacao/oms-perguntas-e-respostas-sobre-o-coronavirus
https://coronavirus.rj.gov.br/o-perguntas-e-respostas-sobre-o-novo-coronavirus/
https://www.bbc.com/portuguese/brasil-51673933
https://www.coronavirus.ms.gov.br/?p=44
https://www.saude.rj.gov.br/noticias/2020/03/perguntas-e-respostas-sobre-o-nov

In [34]:
'www.coronavirus.sc.gov.br' in df_reposi

False